# core

> Helper functions and classes for Conc.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# requirements - numpy polars spacy great_tables, nltk, requests, python-slugify
# dev requirements - nbdev, jupyterlab, jupyterlab-quarto, memory_profiler, line_profiler
# TODO check all necessary
from __future__ import annotations
import re
import os
import logging
import polars as pl
from great_tables import GT
import polars as pl

from memory_profiler import _get_memory


In [ ]:
#| export
PAGE_SIZE = 20
EOF_TOKEN_STR = ' conc-end-of-file-token'

In [ ]:
#| export
DOCUMENTATION_URL = 'https://geoffford.nz/conc'
REPOSITORY_URL = 'https://github.com/polsci/conc'
CITATION_STR = '''If you use Conc in your work, please cite it as follows:'''

In [ ]:
#| exporti
class ConcLogger(logging.Logger):
	""" Custom logger for conc module. """
	def __init__(self, name, level=logging.WARNING, log_file=None):
		super().__init__(name, level)
		self._setup_handler(log_file)
		self.last_memory_usage = None

	def _setup_handler(self, log_file = None):
		console_handler = logging.StreamHandler()
		formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(funcName)s - %(message)s', 
									  datefmt='%Y-%m-%d %H:%M:%S')
		console_handler.setFormatter(formatter)
		self.addHandler(console_handler)

		if log_file is not None:
			file_handler = logging.FileHandler(log_file)
			file_handler.setFormatter(formatter)
			self.addHandler(file_handler)

	def set_state(self, state:str # 'quiet' or 'verbose'
				  ):
		if state == 'quiet':
			level = logging.WARNING
		elif state == 'verbose':
			level = logging.INFO
		else:
			raise ValueError(f"Invalid state: {state}")
		
		self.setLevel(level)

	def memory_usage(self, message = '', init=False):
		if init:
			self.last_memory_usage = None
		usage = _get_memory(-1, 'psutil', include_children=True)
		if self.last_memory_usage is not None:
			difference = usage - self.last_memory_usage
			memory_message = f', memory usage: {usage} MB, difference: {difference} MB'
		else:
			memory_message = f', memory usage: {usage} MB'
		self.info(f"{message}{memory_message}")
		self.last_memory_usage = usage





In [ ]:
#| export
logging.setLoggerClass(ConcLogger)

logger = logging.getLogger(__name__)


In [ ]:
#| export
def set_logger_state(state:str # 'quiet' or 'verbose'
					 ):
	""" Set the state of the conc logger to either 'quiet' or 'verbose' """
	logger.set_state(state)

In [ ]:
#| exporti

toy_data = []
toy_data.append(['1.txt', 'The cat sat on the mat.', 'feline', 'cat'])
toy_data.append(['2.txt', 'The dog sat on the mat.', 'canine', 'dog'])
toy_data.append(['3.txt', 'The cat is meowing.', 'feline', 'cat'])
toy_data.append(['4.txt', 'The dog is barking.', 'canine', 'dog'])
toy_data.append(['5.txt', 'The cat is climbing a tree.', 'feline', 'cat'])
toy_data.append(['6.txt', 'The dog is digging a hole.', 'canine', 'dog'])

source_path = '../test-corpora/source'

In [ ]:
#| export
def create_toy_corpus_sources(source_path:str # path to location of sources for building corpora
							 ):
	""" Create txt files and csv to test build of toy corpus. """

	toy_path = os.path.join(source_path, 'toy')
	if not os.path.exists(toy_path):
		os.makedirs(toy_path, exist_ok=True)
	for row in toy_data:
		with open(f'{source_path}/toy/{row[0]}', 'w', encoding='utf-8') as f:
			f.write(row[1])
	df = pl.DataFrame(toy_data, orient='row', schema=(('source', str), ('text', str), ('category', str), ('species', str)))
	df.write_csv(os.path.join(source_path, 'toy.csv'))
	df.write_csv(os.path.join(source_path, 'toy.csv.gz'))


In [ ]:
#| hide
#| eval: false
create_toy_corpus_sources(source_path)

In [ ]:
#| hide
toy_corpus_df = pl.read_csv(os.path.join(source_path, 'toy.csv'))
GT(toy_corpus_df).tab_options(table_margin_left = 0).show()

source,text,category,species
1.txt,The cat sat on the mat.,feline,cat
2.txt,The dog sat on the mat.,canine,dog
3.txt,The cat is meowing.,feline,cat
4.txt,The dog is barking.,canine,dog
5.txt,The cat is climbing a tree.,feline,cat
6.txt,The dog is digging a hole.,canine,dog


In [ ]:
#| export
def get_nltk_corpus_sources(source_path:str # path to location of sources for building corpora
							 ):
	""" Get nltk corpora as sources for testing. """

	import nltk
	nltk.download('gutenberg')
	nltk.download('brown')
	nltk.download('reuters')
	from nltk.corpus import gutenberg
	from nltk.corpus import reuters
	from nltk.corpus import brown

	def clean_text(text):
		# to match words/punc that followed by /tags
		pattern = re.compile(r"(\S+)(/[^ ]+)") # match non-space followed by / and non-space
		return pattern.sub(r"\1", text)

	if not os.path.exists(source_path):
		os.makedirs(source_path, exist_ok=True)
	if not os.path.exists(f'{source_path}/brown'):
		os.makedirs(f'{source_path}/brown', exist_ok=True)
	brown_path = os.path.join(source_path, 'brown.csv.gz')
	corpus_data = []
	for fileid in brown.fileids():
		corpus_data.append([fileid, clean_text(brown.raw(fileid))])
		with open(f'{source_path}/brown/{fileid}.txt', 'w', encoding='utf-8') as f:
			f.write(clean_text(brown.raw(fileid)))
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(brown_path)

	gutenberg_path = os.path.join(source_path, 'gutenberg.csv.gz')
	corpus_data = []
	for fileid in gutenberg.fileids():
		corpus_data.append([fileid, clean_text(gutenberg.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(gutenberg_path)

	reuters_path = os.path.join(source_path, 'reuters.csv.gz')
	corpus_data = []
	for fileid in reuters.fileids():
		fileid_name = fileid.split('/')[1]
		corpus_data.append([fileid_name, clean_text(reuters.raw(fileid))])
	df = pl.DataFrame(corpus_data, orient='row', schema=(('source', str), ('text', str)))
	df.write_csv(reuters_path)


In [ ]:
#| hide
#| eval: false
get_nltk_corpus_sources(source_path)

[nltk_data] Downloading package gutenberg to /home/geoff/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package brown to /home/geoff/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /home/geoff/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


The texts for the Brown corpus from nltk can be used for testing Conc functionality. The Reuters and Gutenberg corpora are also prepared by `get_nltk_corpus_sources`. Running the function will download the texts and save the texts as a .csv.gz files with columns: source and text. The Brown Corpus is also saved as .txt files to test the Corpus.build_from_texts method.

In [ ]:
#| export
def get_garden_party(source_path: str #path to location of sources for building corpora
					):
    """ Get corpus of The Garden Party by Katherine Mansfield for testing. """
    path = 'https://github.com/ucdh/scraping-garden-party/raw/master/garden-party-corpus.zip'
    import requests
    r = requests.get(path)
    with open(f'{source_path}/garden-party-corpus.zip', 'wb') as f:
        f.write(r.content)
    # convert to .tar and tar.gz files for testing
    import zipfile
    with zipfile.ZipFile(f'{source_path}/garden-party-corpus.zip', 'r') as z:
        z.extractall(f'{source_path}/garden-party-corpus')
    import shutil # make tar.gz
    shutil.make_archive(f'{source_path}/garden-party-corpus', 'gztar', f'{source_path}/garden-party-corpus')
    shutil.move(f'{source_path}/garden-party-corpus.tar.gz', f'{source_path}/garden-party-corpus.tar.gz')
    shutil.make_archive(f'{source_path}/garden-party-corpus', 'tar', f'{source_path}/garden-party-corpus')
    shutil.move(f'{source_path}/garden-party-corpus.tar', f'{source_path}/garden-party-corpus.tar')
    shutil.rmtree(f'{source_path}/garden-party-corpus')
    

In [ ]:
#| hide
#| eval: false
get_garden_party(source_path)

In [ ]:
#| export
def get_large_dataset(source_path: str #path to location of sources for building corpora
                    ):
    """ Get 1m rows of https://huggingface.co/datasets/Eugleo/us-congressional-speeches-subset for testing. """
    df = pl.read_parquet('hf://datasets/Eugleo/us-congressional-speeches-subset/data/train-*.parquet')
    df.sample(1000000).select(['speech_id', 'date', 'speaker', 'chamber', 'state', 'text']).write_csv(f'{source_path}/us-congressional-speeches-subset-1m.csv.gz')
    del df


In [ ]:
# Define the chunk size
chunk_size = 100_000  # Adjust based on your memory constraints

# Lazily load the CSV file
df = pl.scan_csv('../test-corpora/source/us-congressional-speeches-subset-1m.csv.gz')

# Add the new column 'is_empty'
df = df.with_columns(
    (pl.col('text').str.strip_chars().eq('')).alias('is_empty')
)

# get length of is_empty where True
count = df.filter(pl.col("is_empty") == True).collect().height
print(f"Number of empty rows: {count}")

# any empty?
#len(df[df['text'].is_null()])

# # get distribution of date (by year), speaker, chamber, state
# # dates are in iso format - extract year and summarize
# df = df.with_columns(pl.col('date').str.slice(0, 4).alias('year'))
# #df.group_by('year').agg(pl.count('year').alias('count')).sort('year', descending=False).head(200)

# #df.group_by('speaker').agg(pl.count('speaker').alias('count')).sort('count', descending=True).head(20)
# #df.group_by('chamber').agg(pl.count('chamber').alias('count')).sort('count', descending=True).head(20)
# df.group_by('state').agg(pl.count('state').alias('count')).sort('count', descending=True).head(20)

Number of empty rows: 0


In [ ]:
#| export
def create_dataset_sizes(source_path: str, #path to location of sources for building corpora
						sizes: list = [10000, 100000, 200000, 500000] # list of sizes for test data-sets
						):
	""" Create datasets of different sizes for testing. """
	for max_i in sizes:
		max_i_label = int(max_i / 1000)
		df = pl.read_csv(f'{source_path}/us-congressional-speeches-subset-1m.csv.gz')
		df.sample(max_i).write_csv(f'{source_path}/us-congressional-speeches-subset-{max_i_label}k.csv.gz')
		logger.info(f'Creating dataset of {max_i_label}k rows')


In [ ]:
#| hide
#| eval: false
create_dataset_sizes(source_path)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()